In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
plt.style.use('dark_background')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
train_data.describe()

In [ ]:
test_data.describe()

In [ ]:
missing_train = train_data.isnull().sum()
missing_train

In [ ]:
missing_test = test_data.isnull().sum()
missing_test

In [ ]:
sns.countplot('cat0', hue = 'target', data = train_data)

In [ ]:
sns.histplot(data=train_data, x='cont0')

In [ ]:
cat_col = [f'cat{i}' for i in range(19)]
cont_col = [f'cont{i}' for i in range(11)]
columns = cat_col + cont_col

In [ ]:
sns.countplot('target', data=train_data)
plt.title('Balance of Training Data')
plt.show()

In [ ]:
i = 1
plt.figure(figsize=(16,16))
for cat in cat_col:
    plt.subplot(5,4,i)
    sns.countplot(cat, data=train_data)
    i += 1
    plt.tight_layout()

plt.show()

In [ ]:
i = 1
plt.figure(figsize=(16, 26))
for cont in cont_col:
    plt.subplot(11,2,i)
    sns.histplot(train_data[cont])
    i += 1
    
    plt.subplot(11,2,i)
    sns.boxplot(x=train_data[cont])
    i += 1
    
    plt.tight_layout()
    
plt.show()

In [ ]:
full_data = pd.concat([train_data, test_data], axis=0)

full_data = pd.get_dummies(full_data, columns=cat_col)

train = full_data.iloc[:len(train_data), :]
test = full_data.iloc[len(train_data):,:]

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
columns = [column for column in train.columns if column not in ['id', 'target']]

In [ ]:
X = train[columns]
y = train.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=137)

In [ ]:
lgb = LGBMClassifier()
lgb.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=150)
predictions = lgb.predict_proba(X_test)[:,1]

auc = roc_auc_score(y_test, predictions)

print(f'AUC Score : {auc}')

In [ ]:
test = test.drop(labels=['id', 'target'], axis=1)
submission = pd.read_csv('../input/tabular-playground-series-mar-2021/sample_submission.csv')

In [ ]:
submission.target = lgb.predict_proba(test)[:, 1]
submission.to_csv('submission.csv', index = False)

In [ ]:
submission.head()